# Project 3: Poisonous Mushrooms

In this project, you'll investigate properties of mushrooms. This classic dataset contains over 8000 examples, where each describes a mushroom by a variety of features like color, odor, etc., and the target variable is an indicator for whether the mushroom is poisonous. The feature space has been binarized. Look at the feature_names below to see all 126 binary names.

You'll start by running PCA to reduce the dimensionality from 126 down to 2 so that you can easily visualize the data. In general, PCA is very useful for visualization (though sklearn.manifold.tsne is known to produce better visualizations). Recall that PCA is a linear transformation. The 1st projected dimension is the linear combination of all 126 original features that captures as much of the variance in the data as possible. The 2nd projected dimension is the linear combination of all 126 original features that captures as much of the remaining variance as possible. The idea of dense low dimensional representations is crucial to machine learning!

Once you've projected the data to 2 dimensions, you'll experiment with clustering using k-means and density estimation with Gaussian mixture models (GMM). Finally, you'll train a classifier by fitting a GMM for the positive class and a GMM for the negative class, and perform inference by comparing the probabilities output by each model.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please **prepare your own write-up and write your own code**.

In [ ]:
%matplotlib inline

import urllib.request as urllib2 # For python3
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

In [ ]:
MUSHROOM_DATA = 'https://raw.githubusercontent.com/UCB-MIDS/207-Applied-Machine-Learning/master/Data/mushroom.data'
MUSHROOM_MAP = 'https://raw.githubusercontent.com/UCB-MIDS/207-Applied-Machine-Learning/master/Data/mushroom.map'

Load feature names.

In [ ]:
feature_names = []

for line in urllib2.urlopen(MUSHROOM_MAP):
    [index, name, junk] = line.decode('utf-8').split()
    feature_names.append(name)

print('Loaded feature names: ', len(feature_names))
print(feature_names)

Load data. The dataset is sparse, but there aren't too many features, so we'll use a dense representation, which is supported by all sklearn objects.

In [ ]:
X, Y = [], []

for line in urllib2.urlopen(MUSHROOM_DATA):
    items = line.decode('utf-8').split()
    Y.append(int(items.pop(0)))
    x = np.zeros(len(feature_names))
    for item in items:
        feature = int(str(item).split(':')[0])
        x[feature] = 1
    X.append(x)

# Convert these lists to numpy arrays.
X = np.array(X)
Y = np.array(Y)

# Split into train and test data.
train_data, train_labels = X[:7000], Y[:7000]
test_data, test_labels = X[7000:], Y[7000:]

# Check that the shapes look right.
print(train_data.shape, test_data.shape)

### Part 1:

Do a principal components analysis on the data. Show what fraction of the total variance in the training data is explained by the first k principal components, for k in [1, 2, 3, 4, 5, 10, 20, 30, 40, 50].  Also show a lineplot of fraction of total variance vs. number of principal components, for all possible numbers of principal components.

Notes:
* You can use `PCA` to produce a PCA analysis.

In [ ]:
def P1():

### STUDENT START ###
    # Refferance https://towardsdatascience.com/principal-component-analysis-pca-with-scikit-learn-1e84a0c731b0

    # Set principle components (k) and Total Variance array
    k = [1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
    var =[]

    # Calculate Total Variance for K
    for i in k:
        p1_pca = PCA(n_components=i)
        p1_pca.fit_transform(train_data)
        temp = np.sum(p1_pca.explained_variance_ratio_)
        var.append(temp)

        print("Total Variance for k=", i, "is", temp)

    # Plot Fraction of Total Variance vs Number of Principal Components
    j = list(range(126))
    var = []
    for i in j:
        p1_pca = PCA(n_components=i)
        p1_pca.fit_transform(train_data)
        temp = np.sum(p1_pca.explained_variance_ratio_)
        var.append(temp)

    plt.plot(j,var)
    plt.xlabel('Number of Components')
    plt.ylabel('Fraction of Total Variance')

### STUDENT END ###

P1()

### Part 2:

PCA can be very useful for visualizing data. Project the training data down to 2 dimensions and show as a square scatterplot. Show the positive (poisonous) examples in red and the negative (non-poisonous) examples in green. Here's a reference for plotting: http://matplotlib.org/users/pyplot_tutorial.html

Notes:
* You can use `PCA` to produce a PCA analysis.

In [ ]:
def P2():

### STUDENT START ###
    
    # Reduce the data to two dimensions
    p2_pca = PCA(n_components=2)
    x_p2_pca = p2_pca.fit_transform(train_data)


    # How do i determine what is negitive?
    # Set the colors based on labels
    label_color_dict = {1:'red', 0:'green'}

    # Color vector creation
    cvec = [label_color_dict[label] for label in train_labels]

    # Plot
    plt.figure(figsize=(10,7))
    plt.scatter(x = x_p2_pca[:, 0],
                y = x_p2_pca[:, 1],
                marker = 's',
                c=cvec,
                alpha = 0.2)
    plt.title("2D Scatterplot: 29.73% of Variability Captured")
    plt.xlabel("First Principal Component")
    plt.ylabel("Second Principal Component")

    plt.show()


### STUDENT END ###

P2()

### Part 3:

Fit a k-means cluster model with 6 clusters over the 2d projected data. As in part 2, show as a square scatterplot with the positive (poisonous) examples in red and the negative (non-poisonous) examples in green.  For each cluster, mark the centroid and plot a circle that goes through the cluster's example that is most distant from the centroid.

Notes:
* You can use `KMeans` to produce a k-means cluster analysis.
* You can use `linalg.norm` to determine distance (dissimilarity) between observations.

In [ ]:
def P3():

### STUDENT START ###
    # Reduce the data to two dimensions
    p3_pca = PCA(n_components=2)
    train_p3_pca = p3_pca.fit_transform(train_data)

    # Train the Model
    k_means = KMeans(n_clusters = 6)
    k_means.fit_transform(train_p3_pca)

    # Get the Centerpoint of the clusters
    centroids = k_means.cluster_centers_

    # Get Cluster Labels
    cluster_label = k_means.labels_

    # Calculate Radi
    clustered_data = [[0 for i in range(0)] for j in range(6)]
    for index, label in enumerate(cluster_label):
      clustered_data[label].append(np.linalg.norm(train_p3_pca[index] - centroids[label]))

    #Get max of each list
    max_rad = []
    for i in range(len(clustered_data)):
      max_rad.append(max(clustered_data[i]))

    # Set the colors based on labels
    label_color_dict = {1:'red', 0:'green'}

    # Color vector creation
    cvec = [label_color_dict[label] for label in train_labels]

    # Plot
    #plt.figure(figsize=(10,7))
    fig, ax = plt.subplots(figsize=(10,10))
    plt.scatter(x = train_p3_pca[:, 0],
                y = train_p3_pca[:, 1],
                marker = 's',
                c=cvec,
                alpha = 0.2)
    plt.scatter(centroids[:,0],
                centroids[:,1],
                s = 80,
                color = 'black',
                marker = '+')
    for i in range(len(max_rad)):
      circles = plt.Circle((centroids[i][0],
                            centroids[i][1]),
                            max_rad[i],
                            fill = False,
                            color = 'black')
      ax.add_patch(circles)
    plt.title("2D Scatterplot")
    plt.xlabel("First Principal Component")
    plt.ylabel("Second Principal Component")

    plt.show()
    
### STUDENT END ###

P3()

### Part 4:

Fit Gaussian mixture models for the positive (poisonous) examples in your 2d projected data. Vary the number of mixture components from 1 to 4 and the covariance matrix type 'spherical', 'diag', 'tied', 'full' (that's 16 models).  Show square plots of the estimated density contours presented in a 4x4 grid - one row each for a number of mixture components and one column each for a convariance matrix type.  

Notes:
* You can use `GaussianMixture(n_components=..., covariance_type=..., random_state=12345)` to produce a Gaussian mixture model.
* You can use `contour` in combination with other methods to plot contours, like in this example: http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html#example-mixture-plot-gmm-pdf-py
* You can use `contour` without the `norm` and `levels` parameters. 

In [ ]:
def P4():

### STUDENT START ###
    # Define variables provided
    components = [1,2,3,4]
    matrix_type = [ 'spherical', 'diag', 'tied', 'full' ]

    # Get the poisonous data (train_label = 0)
    p4_pca = PCA(n_components=2)
    train_p4_pca = p4_pca.fit_transform(train_data)
    poisonous = []
    for index, label in enumerate(train_labels):
      if label == 1:
        poisonous.append(train_p4_pca[index])

    poisonous = np.array(poisonous)

    # Initialize Plot
    fig, ax = plt.subplots(len(components),
                           len(matrix_type),
                           figsize = (20,25))

    # Build Models
    for i_index, component in enumerate(components):
      for j_index, matrix in enumerate(matrix_type):

        # Train the Model
        gmm = GaussianMixture(n_components = component,
                              covariance_type = matrix,
                              random_state=12345)
        gmm.fit(poisonous)

        # Build Visualization (From documentation)
        x = np.linspace(-3., 3.)
        y = np.linspace(-3., 3.)
        X, Y = np.meshgrid(x, y)
        XX = np.array([X.ravel(), Y.ravel()]).T
        Z = -gmm.score_samples(XX)
        Z = Z.reshape(X.shape)

        # Plot Visualization
        CS = ax[i_index, j_index].contour(X, Y, Z)
        #plt.colorbar(CS, shrink=0.8, extend='both')

        ax[i_index, j_index].scatter(poisonous[:, 0],
                                     poisonous[:, 1],
                                     c = 'gray',
                                     marker = 's',
                                     alpha = 0.2)
        ax[i_index, j_index].set_title('{} Components and {} Cov Type'.format(components[i_index], 
                                                                              matrix_type[j_index]), 
                                       fontsize = 15)
        ax[i_index, j_index].axis('tight')
    
    plt.show()


### STUDENT END ###

P4()

### Part 5:

Fit two Gaussian mixture models, one for the positive examples and one for the negative examples in your 2d projected data. Use 4 mixture components and full convariance for each model.  Predict the test example labels by picking the labels corresponding to the larger of the two models' probabilities.  What is the accuracy of you predictions on the test data?

Notes:
* You can use `GaussianMixture(n_components=..., covariance_type=..., random_state=12345)` to produce a Gaussian mixture model.
* You can use `GaussianMixture`'s `score_samples` method to find the probabilities.

In [ ]:
def P5():

### STUDENT START ###

    # Transform the data sets
    p5_pca = PCA(n_components=2)
    train_p5_pca = p5_pca.fit_transform(train_data)
    test_p5_pca = p5_pca.transform(test_data)

    neg = []
    pos = []

    for index, label in enumerate(train_labels):
      if label == 1:
        neg.append(train_p5_pca[index])
      else:
        pos.append(train_p5_pca[index])

    # Train the models
    gmm = GaussianMixture(n_components = 4,
                              covariance_type = 'full',
                              random_state=12345)
    gmm_pos = gmm.fit(pos)

    gmm = GaussianMixture(n_components = 4,
                              covariance_type = 'full',
                              random_state=12345)
    
    gmm_neg = gmm.fit(neg)

    # Predictions
    predictions_pos = np.exp(gmm_pos.score_samples(test_p5_pca))
    predictions_neg = np.exp(gmm_neg.score_samples(test_p5_pca))
  

    results = (predictions_pos < predictions_neg)
    accuracy = (results == test_labels).sum() / len(test_labels)

    print('Accuracy of prediction of test data is', round(accuracy *100,2),'%')


### STUDENT END ###

P5()

### Part 6:

Run a series of experiments to find the Gaussian mixture model that results in the best accuracy with no more than 50 parameters.  Do this by varying the number of PCA components, the number of GMM components, and the covariance type.

Notes:
* You can use `GaussianMixture(n_components=..., covariance_type=..., random_state=12345)` to produce a Gaussian mixture model.


* For spherical, diag, and full covariance types:
  * number of parameters = (number of parameters per gmm component * number of gmm components - 1) * number of classes
  * number of parameters per gmm component includes all the means plus all the non-zero, non-duplicated values in the covariance matrix plus the mixing weight
  * Each mixing weight parameter indicates how much to weight a particular gmm component; the -1 above accounts for the fact that the mixing weights must sum to 1, so you do not need to include the last mixing weight as its own parameter


* To calculate the number of parameters for tied covariance type:
  * number of parameters = (number of parameters per class - 1) * number of classes
  * number of parameters per class includes all the means and mixing weights for all the gmm components plus all the non-zero, non-duplicated values in the one shared covariance matrix
  * Each mixing weight parameter indicates how much to weight a particular gmm component; the -1 above accounts for the fact that the mixing weights must sum to 1, so you do not need to include the last mixing weight as its own parameter

In [ ]:
def P6():

### STUDENT START ###
    # Define Variables
    n_class = 2
    pca_components = []
    gmm_components = []
    cov_t = []
    accuracies = []
    params = []


    # Outer Loop Varies  Number of PCA Components
    # 24 Max PCA range which results in spherical param <= 50
    for n_pca in range(1, 25): 
        p6_pca = PCA(n_components=n_pca)
        train_p6_pca = p6_pca.fit_transform(train_data)
        test_p6_pca = p6_pca.transform(test_data)

        neg = []
        pos = []

        for index, label in enumerate(train_labels):
          if label == 1:
            neg.append(train_p6_pca[index])
          else:
            pos.append(train_p6_pca[index])

        # Middle loop varies number of GMM components
        # 24 max value for tied param <= 50
        for n_gmm in range(1, 25):

            spher_comp = ((n_pca * n_gmm) + n_gmm) * n_class
            diag_comp = ((n_pca * n_gmm) + (n_pca*n_gmm)) * n_class
            tied_comp = ((n_pca * n_gmm) + n_pca * (n_pca+1) / 2) * n_class
            full_comp = (n_pca*n_gmm + n_pca*(n_pca+1)/2 * n_gmm)*n_class

            # Get accuracy for spherical cov_type <= 50 components
            if (spher_comp <= 50):
                accuracy = get_accuracy(n_gmm, pos, neg, test_p6_pca, 'spherical', test_labels)
                pca_components.append(n_pca)
                gmm_components.append(n_gmm)
                cov_t.append('spherical')
                accuracies.append(accuracy)
                params.append(spher_comp)

            # Get accuracy for diag cov_type <= 50 components
            if (diag_comp <= 50):
                accuracy = get_accuracy(n_gmm, pos, neg, test_p6_pca, 'diag', test_labels)
                pca_components.append(n_pca)
                gmm_components.append(n_gmm)
                cov_t.append('diag')
                accuracies.append(accuracy)
                params.append(diag_comp)

            # Get accuracy for tied cov_type <= 50 components
            if (tied_comp <= 50):
                accuracy = get_accuracy(n_gmm, pos, neg, test_p6_pca, 'tied', test_labels)
                pca_components.append(n_pca)
                gmm_components.append(n_gmm)
                cov_t.append('tied')
                accuracies.append(accuracy)
                params.append(tied_comp)

            # Get accuracy for full cov_type <= 50 components
            if (full_comp <= 50):
                accuracy = get_accuracy(n_gmm, pos, neg, test_p6_pca, 'full', test_labels)
                pca_components.append(n_pca)
                gmm_components.append(n_gmm)
                cov_t.append('full')
                accuracies.append(accuracy)
                params.append(full_comp)
        
    max_value = max(accuracies)
    max_index = accuracies.index(max_value)

    print('Highest Accuracy Model:')
    print('pca_comp =', pca_components[max_index], 
          'gmm_comp =', gmm_components[max_index],
          'cov type =', cov_t[max_index],
          'number of params =', params[max_index],
          'accuracy =', accuracies[max_index]*100, '%')


# Helper function calculates and returns the accuracy of the predictions
# from the provided data and labels
def get_accuracy(n_gmm, pos, neg, test_data, cov_type, test_labels):
    # Assess_model
    gmm_pos = run_gmm(n_gmm, cov_type, pos)
    gmm_neg = run_gmm(n_gmm, cov_type, neg)

    # Calculate Accuracies 
    predictions_pos = np.exp(gmm_pos.score_samples(test_data))
    predictions_neg = np.exp(gmm_neg.score_samples(test_data))
    results = (predictions_pos < predictions_neg)

    return (results == test_labels).sum() / len(test_labels)


# Helper function sets up and executed the model
def run_gmm(n_gmm, cov_type, data):
  gmm = GaussianMixture(n_components = n_gmm, 
                        covariance_type = cov_type, 
                        random_state=12345)
  
  return gmm.fit(data)

### STUDENT END ###

P6()